# Earlier version, was creating artifacts

In [20]:
import cv2
import numpy as np

# Load the foreground (car) and background (street) images
foreground = cv2.imread('car.png', cv2.IMREAD_UNCHANGED)
background = cv2.imread('street.jpg')

# If the foreground is RGBA (with alpha), separate the alpha channel
if foreground.shape[2] == 4:
    alpha_channel = foreground[:, :, 3]  # Get the alpha channel
    foreground = foreground[:, :, :3]    # Remove the alpha channel for blending

# Create a mask from the alpha channel (if available), otherwise create a binary mask
mask = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)
_, mask = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

# Resize the car if it's too large for the street background
bg_height, bg_width = background.shape[:2]
fg_height, fg_width = foreground.shape[:2]

# Ensure the foreground fits within the background
if fg_width > bg_width or fg_height > bg_height:
    scale_factor = min(bg_width / fg_width, bg_height / fg_height)
    foreground = cv2.resize(foreground, (int(fg_width * scale_factor), int(fg_height * scale_factor)))
    mask = cv2.resize(mask, (int(fg_width * scale_factor), int(fg_height * scale_factor)))

# Define the position where the car should be placed on the street
# Let's position it in the middle of the street
center_x = bg_width // 2
center_y = int(bg_height * 0.75)  # Lower part of the street
center = (center_x, center_y)

# Perform Poisson blending
blended_image = cv2.seamlessClone(foreground, background, mask, center, cv2.NORMAL_CLONE)

# Show the result
cv2.imshow('Blended Image', blended_image)
cv2.imwrite('blended_car_on_street.jpg', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

# Updated code, no artifacts

In [23]:
import cv2
import numpy as np

# Load the images
foreground = cv2.imread('car.png', cv2.IMREAD_UNCHANGED)
background = cv2.imread('street.jpg')

# Check if foreground has an alpha channel (transparency)
if foreground.shape[2] == 4:
    # Extract the alpha channel (used as a mask)
    alpha_channel = foreground[:, :, 3]
    mask = alpha_channel
    # Convert the image to BGR (remove alpha channel for blending)
    foreground = foreground[:, :, :3]
else:
    # Create a mask from the brightness of the image if no alpha channel is present
    gray_foreground = cv2.cvtColor(foreground, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(gray_foreground, 1, 255, cv2.THRESH_BINARY)

# Resize the foreground if it's too large for the background
bg_height, bg_width = background.shape[:2]
fg_height, fg_width = foreground.shape[:2]

if fg_width > bg_width or fg_height > bg_height:
    scale = min(bg_width / fg_width, bg_height / fg_height)
    foreground = cv2.resize(foreground, (int(fg_width * scale), int(fg_height * scale)))
    mask = cv2.resize(mask, (int(fg_width * scale), int(fg_height * scale)))

# Position the car on the street
center = (bg_width // 2, int(bg_height * 0.75))

# Perform Poisson blending
blended_image = cv2.seamlessClone(foreground, background, mask, center, cv2.NORMAL_CLONE)

# Show and save the result
cv2.imshow('Blended Image', blended_image)
cv2.imwrite('blended_car_on_street_fixed.jpg', blended_image)
cv2.waitKey(0)
cv2.destroyAllWindows()
